In [6]:
import os
import pandas as pd

def resample(df):
    df_wide = df.pivot_table(values='wc', index='dateTime', columns=['side', 'depth'], aggfunc='mean').reset_index()
    df_wide.columns = [f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col for col in df_wide.columns]
    df_wide.rename(columns={df_wide.columns[0]: 'dateTime'}, inplace=True)
    df_modified = pd.merge(df.drop(['wc', 'side', 'depth'], axis=1).drop_duplicates(), df_wide, on='dateTime', how='left')
    df_modified.set_index('dateTime', inplace=True)
    numeric_df = df_modified.select_dtypes(include=['number']).resample('D').mean()
    return numeric_df

input_dir_path = "data/sensor_data_clean/"
output_dir_path = "data/daily_data/"
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

dfs = []

for file_name in os.listdir(input_dir_path):
    input_file_path = os.path.join(input_dir_path, file_name)
    df = pd.read_csv(input_file_path, na_values="NA")
    df['dateTime'] = pd.to_datetime(df['dateTime'])
    df_daily = resample(df)
    dfs.append(df_daily)
    output_file_path = os.path.join(output_dir_path, file_name.replace('.csv', '_daily.csv'))
    df_daily.to_csv(output_file_path, sep=";")

df_all = pd.concat(dfs).sort_index()
output_all_file_path = os.path.join(output_dir_path, "all_data_daily.csv")
df_all.to_csv(output_all_file_path, sep=";")